<a href="https://colab.research.google.com/github/drojassa/CUDA/blob/main/Sumar_vectores_CUDA_en_Google_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Ejecución de código CUDA en GOOGLE COLAB**


1. En el Entorno de ejecución se escoge la opción, Acelerador por GPU

2. Se carga Preámbulo para verificar GPU asignada

In [ ]:
# Mostrar si hay un toolkit CUDA instalado
!ls /usr/local
# Para mostrar si esta el comando nvcc disponible
!which nvcc
# Mostrar la tarjeta nvidia adjudicada
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

bin   cuda-11	 etc	_gcs_config_ops.so  lib        man   share  xgboost
cuda  cuda-11.2  games	include		    licensing  sbin  src
/usr/local/cuda/bin/nvcc
Wed Nov  9 00:41:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   47C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+--------------

3. Carga el código a utilizar en la carpeta de archivos, mediante el comando %%writefile RunCode.cu, Con el encabezado RunCode.cu. El código va a trabajar se pega debajo de: %%writefile RunCode.cu

In [ ]:
#Escribe el archivo .cu que se vaya a utilizar

%%writefile RunCode.cu

#include <iostream>
#include <fstream>
#include <cmath>

using namespace std;

#define Lx 16
#define Nx 8
const int Mx=(Lx+Nx-1)/Nx;

//----------Programa del device-----------
//----------Kernels-----------------------
__global__ void AddTwoVectors(float *d_a, float *d_b, float *d_c){
    //Qué tarea me toca?
    int ix; ix=blockIdx.x*blockDim.x+threadIdx.x;
    d_c[ix]=d_a[ix]+d_b[ix];
}

//----------Programa del host-------------
int main(void){
    int ix;
    //Declarar todas las variables por duplicado
    //-----en el host
    float h_a[Lx],h_b[Lx],h_c[Lx];
    //-----en el device
    float *d_a; cudaMalloc((void**) &d_a, Lx*sizeof(float));
    float *d_b; cudaMalloc((void**) &d_b, Lx*sizeof(float));
    float *d_c; cudaMalloc((void**) &d_c, Lx*sizeof(float));

    //LLenar los datos en el host
    for (ix=0;ix<Lx;ix++){
        h_a[ix]=ix;h_b[ix]=2*ix;
    }

    //Enviar al device
    cudaMemcpy(d_a,h_a,Lx*sizeof(float),cudaMemcpyHostToDevice);
    cudaMemcpy(d_b,h_b,Lx*sizeof(float),cudaMemcpyHostToDevice);

    //correr en el device
    dim3 ThreadsPerBlock(Nx,1,1);
    dim3 BlocksPerGrid(Mx,1,1);
    AddTwoVectors<<< BlocksPerGrid,ThreadsPerBlock >>>(d_a,d_b,d_c);

    //Devolver el resultado al host
    cudaMemcpy(h_c,d_c,Lx*sizeof(float),cudaMemcpyDeviceToHost);

    //Imprimir resultados
    for (ix=0;ix<Lx;ix++)
        cout<<h_c[ix]<<endl;
    cudaFree(d_a);cudaFree(d_b);cudaFree(d_c);
    return 0;
}


Overwriting RunCode.cu


4. Compila el archivo .cu

*   Las banderas colocadas en la compilación son necesarias cuando la tarjeta adjudicada es una K80.
*   Si todo esta bien el resultado de esta prueba debe ser 8.

In [ ]:
# Compila el archivo .cu. La bandera es necesaria si usa una tarjeta Tesla K80
!nvcc -arch=sm_37 -gencode=arch=compute_37,code=sm_37 RunCode.cu -o Ejecutar
!./Ejecutar

nvcc warning : The 'compute_35', 'compute_37', 'compute_50', 'sm_35', 'sm_37' and 'sm_50' architectures are deprecated, and may be removed in a future release (Use -Wno-deprecated-gpu-targets to suppress warning).
0
3
6
9
12
15
18
21
24
27
30
33
36
39
42
45
